In [5]:
import os
import sys
base_dir = os.path.join("/home/jovyan/work/Corona/COVID-19")
sys.path.append(base_dir)
import model as md
import pandas as pd

In [6]:
#NewsExtractor(2)
def createJson(date, day, json=False):
    news_reader = md.NewsReader(data_dir="data", month=date, date="{}-{}".format(date, day), string_filter=True)
    repartition = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["body"].unique()
    date_of_info = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["date"].unique()
    print(date_of_info)
    if len(repartition)>0:
        table = "Aktív fertőzöttek száma{}".format(repartition[0].split("Aktív fertőzöttek száma")[1]).split("\n\n\n\n\n")
        df = pd.DataFrame(table, columns=["full"])
        df["full"] = df["full"].str.split("\n\n\n")
        df["len"] = df["full"].apply(lambda cell: len(cell))
        df = df[df["len"]==3].copy()
        df['region'], df['active'], df['death'] = zip(*df["full"])
        df['date'] = "{}-{}".format(date, day)
        df[['date', 'region', 'active', 'death']].to_json('repartition/{}-{}.json'.format(date, day), orient='records')
        return df[['date', 'region', 'active', 'death']]

In [12]:
createJson("2021-02", "12", True)

INFO:root:2021-02-12 08:44:00
INFO:root:https://koronavirus.gov.hu/cikkek/1860-fovel-emelkedett-beazonositott-fertozottek-szama-es-elhunyt-99-beteg
INFO:root:
1 860 újabb magyar állampolgárnál mutatták ki az új koronavírus-fertőzést (COVID-19), ezzel 383 735 főre nőtt a hazánkban beazonosított fertőzöttek száma. Elhunyt 99 többségében idős, krónikus beteg, így az elhunytak száma 13 543 főre emelkedett. A gyógyultak száma folyamatosan nő, jelenleg 289 520 fő, az aktív fertőzöttek száma 80 672 főre csökkent. 3828 koronavírusos beteget ápolnak kórházban, közülük 299-en vannak lélegeztetőgépen. Eddig 310 448 fő kapott oltást, közülük 120 649 fő már a második oltását is megkapta. Az e heti oltási terv alapján a háziorvosok többsége már oltja a védőoltásra regisztrált legidősebbeket (Pfizer-vakcinával) és a 60 év alatti krónikus betegeket (AstraZeneca-vakcinával). Emellett ma négy budapesti kórházi oltóponton is oltani kezdik a bizonyos kockázati krónikus betegséggel nem rendelkező idősebbek

INFO:root:2021-02-12 10:10:00
INFO:root:https://koronavirus.gov.hu/cikkek/orban-ha-kinai-vakcinaval-elindulhat-az-oltas-husvetig-minden-regisztralt-beolthato
INFO:root:
Ha a kínai vakcinával is elkezdődhet az oltás, húsvétig minden eddig regisztráltat be lehet oltani a koronavírus ellen - mondta Orbán Viktor pénteken a Kossuth rádió Jó reggelt, Magyarország! című műsorában. A nyitásról tervezett online konzultáció ugyanazon az internetes felületen - a vakcinainfo.gov.hu honlapon - zajlik majd, mint a regisztráció.


Tartanunk kell a szabályokat, de nem szükségesek további szigorítások
A kormányfő jó hírnek nevezte, hogy már mintegy 700 ezren védettek Magyarországon: 310 ezren már legalább egy oltást megkaptak, 383 ezren pedig már igazoltan átestek a betegségen.
Rossz hír azonban, hogy a fertőzések számának csökkenése megállt, újra emelkedik - mutatott rá, ismertetve a friss adatokat, amelyek szerint az új koronavírus-fertőzöttek száma 1860, elhunytak 99-en, kórházban 3828-an fekszenek,